# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import os
 
# third-party library
from torch.autograd import Variable
import matplotlib.pyplot as plt

BATCH_SIZE = 128
NUM_EPOCHS = 5
LR=0.01

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [6]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = Variable(torch.unsqueeze(test_data.test_data, dim=1), volatile=True).type(torch.FloatTensor)[:2000]/255.   # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.test_labels[:2000]

E:\anacond3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]


Then, we define the model, object function and optimizer that we use to classify.

In [ ]:
class SimpleNet(nn.Module):

# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # input height
                out_channels=16,            # n_filters
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (16, 28, 28)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # input shape (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # output shape (32, 14, 14)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2),                # output shape (32, 7, 7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)   # fully connected layer, output 10 classes
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output, x    # return x for visualization
    
model = SimpleNet()

# TODO:define loss function and optimiter
#optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
#loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

Next, we can start to train and evaluate!

In [9]:
# train and evaluate
step=1
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):# TODO:forward + backward + optimize
        step=step+1
        b_x = Variable(images)   # batch x
        b_y = Variable(labels)   # batch y
 
        output = model(b_x)[0]               # cnn output
        
        pred_y = torch.max(output, 1)[1].data.squeeze()
        loss = criterion(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        accuracy = sum(pred_y == b_y)
        if step % 50 == 0:
            test_output, last_layer = model(test_x)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == test_y).sum().numpy() / test_y.size(0)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data[0], '|test accuracy: %.2f' % accuracy)
            
            print('Epoch: ', epoch, '|train accuracy: %.2f' % accuracy)
        
         
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    
    
    
    






  0%|                                                                                          | 0/468 [00:00<?, ?it/s]



  0%|▏                                                                                 | 1/468 [00:00<02:10,  3.57it/s]



  0%|▎                                                                                 | 2/468 [00:00<02:07,  3.67it/s]



  1%|▌                                                                                 | 3/468 [00:00<02:07,  3.66it/s]



  1%|▋                                                                                 | 4/468 [00:01<02:09,  3.58it/s]



  1%|▉                                                                                 | 5/468 [00:01<02:08,  3.62it/s]



  1%|█                                                                                 | 6/468 [00:01<02:09,  3.58it/s]



  1%|█▏                                                                                | 7/468 [00:01<02:12,  3.49it/s]



  2%|█▍     

Epoch:  0 | train loss: 0.1532 |test accuracy: 0.92
Epoch:  0 |train accuracy: 0.92






 10%|████████▍                                                                        | 49/468 [00:15<05:05,  1.37it/s]



 11%|████████▋                                                                        | 50/468 [00:15<04:08,  1.68it/s]



 11%|████████▊                                                                        | 51/468 [00:16<03:32,  1.96it/s]



 11%|█████████                                                                        | 52/468 [00:16<03:07,  2.22it/s]



 11%|█████████▏                                                                       | 53/468 [00:16<02:49,  2.44it/s]



 12%|█████████▎                                                                       | 54/468 [00:16<02:35,  2.66it/s]



 12%|█████████▌                                                                       | 55/468 [00:17<02:28,  2.79it/s]



 12%|█████████▋                                                                       | 56/468 [00:17<02:21,  2.92it/s]



 12%|███████

Epoch:  0 | train loss: 0.0842 |test accuracy: 0.95
Epoch:  0 |train accuracy: 0.95






 21%|█████████████████▏                                                               | 99/468 [00:32<04:37,  1.33it/s]



 21%|█████████████████                                                               | 100/468 [00:32<03:46,  1.62it/s]



 22%|█████████████████▎                                                              | 101/468 [00:32<03:11,  1.92it/s]



 22%|█████████████████▍                                                              | 102/468 [00:32<02:45,  2.21it/s]



 22%|█████████████████▌                                                              | 103/468 [00:33<02:29,  2.43it/s]



 22%|█████████████████▊                                                              | 104/468 [00:33<02:16,  2.66it/s]



 22%|█████████████████▉                                                              | 105/468 [00:33<02:09,  2.81it/s]



 23%|██████████████████                                                              | 106/468 [00:34<02:01,  2.98it/s]



 23%|███████

...





 70%|███████████████████████████████████████████████████████▉                        | 327/468 [01:26<01:13,  1.92it/s]



 70%|████████████████████████████████████████████████████████                        | 328/468 [01:26<01:00,  2.33it/s]



 70%|████████████████████████████████████████████████████████▏                       | 329/468 [01:26<00:49,  2.79it/s]



 71%|████████████████████████████████████████████████████████▍                       | 330/468 [01:26<00:43,  3.16it/s]



 71%|████████████████████████████████████████████████████████▌                       | 331/468 [01:27<00:38,  3.51it/s]



 71%|████████████████████████████████████████████████████████▊                       | 332/468 [01:27<00:35,  3.79it/s]



 71%|████████████████████████████████████████████████████████▉                       | 333/468 [01:27<00:33,  4.08it/s]



 71%|█████████████████████████████████████████████████████████                       | 334/468 [01:27<00:31,  4.24it/s]



 72%|███████

Epoch:  4 | train loss: 0.0469 |test accuracy: 0.94
Epoch:  4 |train accuracy: 0.94






 81%|████████████████████████████████████████████████████████████████▍               | 377/468 [01:38<01:00,  1.49it/s]



 81%|████████████████████████████████████████████████████████████████▌               | 378/468 [01:39<00:49,  1.84it/s]



 81%|████████████████████████████████████████████████████████████████▊               | 379/468 [01:39<00:39,  2.25it/s]



 81%|████████████████████████████████████████████████████████████████▉               | 380/468 [01:39<00:34,  2.55it/s]



 81%|█████████████████████████████████████████████████████████████████▏              | 381/468 [01:39<00:30,  2.84it/s]



 82%|█████████████████████████████████████████████████████████████████▎              | 382/468 [01:40<00:27,  3.16it/s]



 82%|█████████████████████████████████████████████████████████████████▍              | 383/468 [01:40<00:26,  3.23it/s]



 82%|█████████████████████████████████████████████████████████████████▋              | 384/468 [01:40<00:24,  3.43it/s]



 82%|███████

Epoch:  4 | train loss: 0.0590 |test accuracy: 0.95
Epoch:  4 |train accuracy: 0.95






 91%|████████████████████████████████████████████████████████████████████████▉       | 427/468 [01:51<00:27,  1.48it/s]



 91%|█████████████████████████████████████████████████████████████████████████▏      | 428/468 [01:52<00:22,  1.79it/s]



 92%|█████████████████████████████████████████████████████████████████████████▎      | 429/468 [01:52<00:17,  2.20it/s]



 92%|█████████████████████████████████████████████████████████████████████████▌      | 430/468 [01:52<00:15,  2.45it/s]



 92%|█████████████████████████████████████████████████████████████████████████▋      | 431/468 [01:52<00:13,  2.82it/s]



 92%|█████████████████████████████████████████████████████████████████████████▊      | 432/468 [01:53<00:11,  3.20it/s]



 93%|██████████████████████████████████████████████████████████████████████████      | 433/468 [01:53<00:10,  3.40it/s]



 93%|██████████████████████████████████████████████████████████████████████████▏     | 434/468 [01:53<00:09,  3.65it/s]



 93%|███████

#### Q5:
Please print the training and testing accuracy.